# Prediction of energy consumption 

## Importing packages and reading data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats
from sklearn.preprocessing import LabelBinarizer,RobustScaler, StandardScaler, MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, ElasticNet, Ridge
from sklearn.svm import LinearSVR

data = pd.read_csv("data\Steel_industry_data.csv")

## Data preprocessing and analysis

Data preview

In [ ]:
data.head()

Shape of data

In [ ]:
data.shape

### Information about the dataset

Description of variables:
| **Variable**                    | **Type**    | **Measurement**                       |
|---------------------------------|-------------|---------------------------------------|
| Industry Energy Consumption     | Continuous  | kWh                                   |
| Lagging Current reactive power  | Continuous  | kVarh                                 |
| Leading Current reactive power  | Continuous  | kVarh                                 |
| tCO2(CO2)                       | Continuous  |  ppm                                  |
| Lagging Current power factor    | Continuous  | %                                     |
| Leading Current Power factor    | Continuous  | %                                     |
| Number of Seconds from midnight | Continuous  | s                                     |
| Week status                     | Categorical | Weekend (0) or a Weekday(1)           |
| Day of week                     | Categorical | Sunday, Monday...Saturday             |
| Load Type                       | Categorical | Light Load, Medium Load, Maximum Load |
|                                 |             |                                       |

In [ ]:
data.info()

### Checking missing values

In [ ]:
data.isnull().sum()

### Identify duplicates

In [ ]:
dups = data.duplicated()
data[dups]

### Identify mistyped data

Check numeric

In [ ]:
numeric = data.applymap(lambda x: isinstance(x, (int, float)))['Usage_kWh']
data[~numeric]

Check string

In [ ]:
strings = data.applymap(lambda x: isinstance(x, (str)))['Day_of_week']
data[~strings]

### Columns info

Column 'WeekStatus' values:

In [ ]:
data['WeekStatus'].value_counts()

Column 'Load_Type' values:

In [ ]:
data['Load_Type'].value_counts()

Mean value

In [ ]:
data.mean(numeric_only=True)

Median value

In [ ]:
data.median(numeric_only=True)

In [ ]:
data['Usage_kWh'].plot(kind="density", figsize=(7,7))
plt.vlines(data['Usage_kWh'].mean(), ymin = 0, ymax = 0.06, linewidth = 5.0)
plt.vlines(data['Usage_kWh'].median(), ymin=0, ymax=0.06, linewidth=2.0, color="red")

Mode value

In [ ]:
data.mode()

Variance

In [ ]:
data['Usage_kWh'].var()

Standard deviation

In [ ]:
data['Usage_kWh'].std()

Percentiles

In [ ]:
percentiles = np.percentile(data['Usage_kWh'], [0, 25, 50, 75, 100])
percentiles

'Usage_kWh' column description:

In [ ]:
desc_stat = scipy.stats.describe(data['Usage_kWh'], ddof = 1, bias = False)
desc_stat

Data description

In [ ]:
data.describe()

Usage_kWh vs Day of Week

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(10,8))
ax = sns.boxplot(x='Usage_kWh', data=data, y='Day_of_week', orient="h")

Correlation map

In [ ]:
# correlation = data.corr(numeric_only=True)
# plt.figure(figsize=(7,7))
# sns.heatmap(correlation, cbar=True, square=True, fmt='.1f', annot=True, annot_kws={'size':8}, cmap='Blues')


Day of Week

In [ ]:
sns.displot(data['Day_of_week'], height=5, aspect=2)

Boxplots

In [ ]:
fig = plt.figure(figsize =(10, 7))
ax = fig.add_axes([0, 0, 1, 1])
ax.boxplot([data['Usage_kWh'], data['Lagging_Current_Power_Factor'], data['Leading_Current_Power_Factor'], data['Lagging_Current_Reactive.Power_kVarh'], data['Leading_Current_Reactive_Power_kVarh']])

### Feature Engineering

Encoding Nominal Categorial Data

In [ ]:
one_hot = LabelBinarizer()

encoded_weekstatus = one_hot.fit_transform(data['WeekStatus'])
data['WeekStatus'] = one_hot.fit_transform(data['WeekStatus'])
print(one_hot.classes_)
print(encoded_weekstatus)

encoded_load_type = one_hot.fit_transform(data['Load_Type'])
data['Load_Type'] = one_hot.fit_transform(data['Load_Type'])

print(one_hot.classes_)
encoded_load_type

Encoding Ordinal Categorial Data

In [ ]:
scale_mapper = {"Monday": 1,
                "Tuesday": 2,
                "Wednesday": 3,
                "Thursday": 4,
                "Friday": 5,
                "Saturday": 6,
                "Sunday": 7}

encoded_day_of_week = data['Day_of_week'].replace(scale_mapper)
data['Day_of_week'] = data['Day_of_week'].replace(scale_mapper)

print(data['Day_of_week'].value_counts())
#encoded_day_of_week.value_counts()

Correlation map after encoding categorical data

In [ ]:
# correlation = data.corr(numeric_only=True)
# plt.figure(figsize=(7,7))
# sns.heatmap(correlation, cbar=True, square=True, fmt='.1f', annot=True, annot_kws={'size':8})

In [ ]:
data = data.loc[:, data.columns != 'date']

## Model building and Evaluation

### Testing different Regression Models with all features

In [ ]:
features, target = data.loc[:, data.columns != 'Usage_kWh'], data['Usage_kWh']
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=0)

training_score = []

dummy = DummyRegressor(strategy="mean")
dummy.fit(features_train, target_train)
d_score = dummy.score(features_test, target_test)


clf = DummyRegressor(strategy='constant', constant=20)
clf.fit(features_train, target_train)
clf_score = clf.score(features_train, target_train)

lr = LinearRegression()
lr.fit(features_train, target_train)

training_score.append(lr.score(features_train, target_train))

lasso = Lasso(alpha=0.1)
lasso.fit(features_train, target_train)

training_score.append(lasso.score(features_train, target_train))

en = ElasticNet(alpha=0.1)
en.fit(features_train, target_train)

training_score.append(en.score(features_train, target_train))

rlr = Ridge(alpha=0.1)
rlr.fit(features_train, target_train)

training_score.append(rlr.score(features_train, target_train))

scaler = MinMaxScaler()
features_train = scaler.fit_transform(features_train)
features_test = scaler.transform(features_test)
svr_max_scaler = LinearSVR()
svr_max_scaler.fit(features_train, target_train)

training_score.append(svr_max_scaler.score(features_train, target_train))

scaler = StandardScaler()
features_train = scaler.fit_transform(features_train)
features_test = scaler.transform(features_test)
svr_standard_scaler = LinearSVR()
svr_standard_scaler.fit(features_train, target_train)

training_score.append(svr_standard_scaler.score(features_train, target_train))


In [ ]:
def show_results(score_list: list, labels: list):
    train_score = pd.DataFrame(data = score_list, columns = ['Training_R2'])
    train_score.index = labels
    train_score = (train_score * 100).round(4)
    plt.xticks(rotation=90)
    plt.grid(visible=None)
    plt.scatter(x=train_score.index, y=train_score['Training_R2'], c=train_score['Training_R2'], cmap='Dark2_r')
    for i in range(len(train_score['Training_R2'])):
        plt.annotate(str(train_score['Training_R2'][i]), (i, train_score['Training_R2'][i]), textcoords="offset points", xytext=(0,10))

Training Score

In [ ]:
models = ['LR', 'LASSO', 'ELNT', 'RIDGE', 'SVR_MAX_SCALER', 'SVR_NORMAL_SCALER']
show_results(score_list=training_score, labels=models)

Describing data after removing row with all zeroes

In [ ]:
data = data[data['Usage_kWh'] != 0]

data.describe()

### Different Scalers for Linear Regression

In [ ]:
features, target = data.loc[:, data.columns != 'Usage_kWh'], data['Usage_kWh']
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=0)

testing_score = []
scaler = RobustScaler()
features_train = scaler.fit_transform(features_train)
features_test = scaler.transform(features_test)
model = LinearRegression()
model.fit(features_train, target_train)
testing_score.append(model.score(features_test, target_test))

scaler = MinMaxScaler()
X_train = scaler.fit_transform(features_train)
X_test = scaler.transform(features_test)
model = LinearRegression()
model.fit(features_train, target_train)
testing_score.append(model.score(features_test, target_test))

scaler = StandardScaler()
features_train = scaler.fit_transform(features_train)
features_test = scaler.transform(features_test)
model = LinearRegression()
model.fit(features_train, target_train)
testing_score.append(model.score(features_test, target_test))

nof_list=np.arange(1,9)
high_score=0

Score of LR Models with different scalers

In [ ]:
labels = ['Robust_Scaler', 'MinMax_Scaler', 'StandardScaler']
show_results(score_list=testing_score, labels=labels)

<h3>RFE Method for Feature Selection</h3>

In [ ]:
def rfe_method(model):
    nof_list=np.arange(1,9)            
    high_score=0

    nof=0           
    score_list =[]
    for n in range(len(nof_list)):
        features, target = data.loc[:, data.columns != 'Usage_kWh'], data['Usage_kWh']
        features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=0)

        rfe = RFE(estimator=model, n_features_to_select=nof_list[n])
        features_train_rfe = rfe.fit_transform(features_train,target_train)
        features_test_rfe = rfe.transform(features_test)
        model.fit(features_train_rfe, target_train)
        score = model.score(features_test_rfe, target_test)
        score_list.append(score)
        if(score>high_score):
            high_score = score
            nof = nof_list[n]
    print("Optimum number of features: %d" %nof)
    print("Score with %d features: %f" % (nof, high_score))

    features, target = data.loc[:, data.columns != 'Usage_kWh'], data['Usage_kWh']
    cols = list(features.columns)
    rfe = RFE(estimator=model, n_features_to_select=nof)             
    rfe = rfe.fit(features, target)  
    print(rfe.get_feature_names_out())
    return features[features.columns[rfe.support_]]

<h3>Grid Search for Hyperparameters</h3>

In [ ]:
def gridsearch(model, space, scoring='neg_mean_absolute_error', cv=10):
    grid_search = GridSearchCV(model, space, scoring=scoring, n_jobs=-1, cv=cv)
    grid_result = grid_search.fit(features_train, target_train)
    
    print('Grid Search - Best Score: %s' % grid_result.best_score_)
    print('Grid Search - Best Hyperparameters: %s' % grid_result.best_params_)
    return grid_result.best_params_

<h3>Random Search for Hyperparameters</h3>

In [ ]:
def randomsearch(model, space, scoring='neg_mean_absolute_error'):
    randomized_search = RandomizedSearchCV(model, space, scoring=scoring)
    randomized_result = randomized_search.fit(features_train, target_train)
    print('Randomized Search - Best Score: %s' % randomized_result.best_score_)
    print('Randomized Search - Best Hyperparameters: %s' % randomized_result.best_params_)
    return randomized_result.best_params_

<h3>Linear Regression</h3>

In [ ]:
model = LinearRegression()

space = dict()
space['fit_intercept'] = [True, False]
hyperparams_gridsearch = gridsearch(model, space)
hyperparams_randomsearch = randomsearch(model, space)

if hyperparams_gridsearch['fit_intercept'] == hyperparams_randomsearch['fit_intercept']:
    model.fit_intercept = hyperparams_gridsearch['fit_intercept']

    selected_features = rfe_method(model)

    features, target = selected_features, data['Usage_kWh']
    features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=0)

    model.fit(features_train, target_train)
    train_score = model.score(features_train, target_train)
    test_score = model.score(features_test, target_test)

    print("The train score for LR model is {}".format(train_score))
    print("The test score for LR model is {}".format(test_score))
else:
    model.fit_intercept = hyperparams_gridsearch['fit_intercept']

    selected_features = rfe_method(model)

    features, target = selected_features, data['Usage_kWh']
    features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=0)

    model.fit(features_train, target_train)
    train_score = model.score(features_train, target_train)
    test_score = model.score(features_test, target_test)

    print("GridSearch: The train score for LR model is {}".format(train_score))
    print("GridSearch: The test score for LR model is {}".format(test_score))

    model.fit_intercept = hyperparams_randomsearch['fit_intercept']

    selected_features = rfe_method(model)

    features, target = selected_features, data['Usage_kWh']
    features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=0)

    model.fit(features_train, target_train)
    train_score = model.score(features_train, target_train)
    test_score = model.score(features_test, target_test)

    print("RandomSearch: The train score for LR model is {}".format(train_score))
    print("RandomSearch: The test score for LR model is {}".format(test_score))

<h3>Ridge Regression</h3>

In [278]:
model = Ridge()

space = dict()
space['solver'] = ['svd', 'lsqr', 'sag']
space['alpha'] = [0.0001, 0.001, 0.01, 0.1, 1, 10]
space['fit_intercept'] = [True, False]
hyperparams_gridsearch = gridsearch(model, space)

model.fit_intercept = hyperparams_gridsearch['fit_intercept']
model.alpha = hyperparams_gridsearch['alpha']
model.solver = hyperparams_gridsearch['solver']

selected_features = rfe_method(model)

features, target = selected_features, data['Usage_kWh']
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=0)

model.fit(features_train, target_train)
train_score = model.score(features_train, target_train)
test_score = model.score(features_test, target_test)

print("The train score for ridge model (alpha=" + str(model.alpha) + ") is {}".format(train_score))
print("The train score for ridge model (alpha=" + str(model.alpha) + ") is {}".format(test_score))

Grid Search - Best Score: -2.5314377952569513
Grid Search - Best Hyperparameters: {'alpha': 0.0001, 'fit_intercept': True, 'solver': 'svd'}
Optimum number of features: 8
Score with 8 features: 0.978820
['Lagging_Current_Reactive.Power_kVarh'
 'Leading_Current_Reactive_Power_kVarh' 'CO2(tCO2)'
 'Lagging_Current_Power_Factor' 'Leading_Current_Power_Factor'
 'WeekStatus' 'Day_of_week' 'Load_Type']
The train score for ridge model (alpha=0.0001) is 0.9810272389259522
The train score for ridge model (alpha=0.0001) is 0.9788204516050691


<h3>Lasso Regression</h3>

In [280]:
model = Lasso()

space = dict()
space['alpha'] = [0.0001, 0.001, 0.01, 0.1, 1, 10]
space['fit_intercept'] = [True, False]
hyperparams_gridsearch = gridsearch(model, space)

model.fit_intercept = hyperparams_gridsearch['fit_intercept']
model.alpha = hyperparams_gridsearch['alpha']

selected_features = rfe_method(model)

features, target = selected_features, data['Usage_kWh']
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=0)

model.fit(features_train, target_train)
train_score = model.score(features_train, target_train)
test_score = model.score(features_test, target_test)

print("The train score for lasso model (alpha=" + str(model.alpha) + ") is {}".format(train_score))
print("The train score for lasso model (alpha=" + str(model.alpha) + ") is {}".format(test_score))

Grid Search - Best Score: -2.5347275038150663
Grid Search - Best Hyperparameters: {'alpha': 0.0001, 'fit_intercept': True}
Optimum number of features: 8
Score with 8 features: 0.978826
['Lagging_Current_Reactive.Power_kVarh'
 'Leading_Current_Reactive_Power_kVarh' 'CO2(tCO2)'
 'Lagging_Current_Power_Factor' 'Leading_Current_Power_Factor'
 'WeekStatus' 'Day_of_week' 'Load_Type']
The train score for lasso model (alpha=0.0001) is 0.9810268960946551
The train score for lasso model (alpha=0.0001) is 0.9788259551480127


<h3>ElasticNet</h3>

In [281]:
model = ElasticNet()

space = dict()
space['alpha'] = [0.0001, 0.001, 0.01, 0.1, 1, 10]
space['fit_intercept'] = [True, False]
hyperparams_gridsearch = gridsearch(model, space)

model.fit_intercept = hyperparams_gridsearch['fit_intercept']
model.alpha = hyperparams_gridsearch['alpha']

selected_features = rfe_method(model)

features, target = selected_features, data['Usage_kWh']
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=0)

model.fit(features_train, target_train)
train_score = model.score(features_train, target_train)
test_score = model.score(features_test, target_test)

print("The train score for Elastic Net model (alpha=" + str(model.alpha) + ") is {}".format(train_score))
print("The train score for Elastic Net model (alpha=" + str(model.alpha) + ") is {}".format(test_score))

Grid Search - Best Score: -5.0729784359929395
Grid Search - Best Hyperparameters: {'alpha': 0.0001, 'fit_intercept': False}
Optimum number of features: 3
Score with 3 features: 0.949066
['CO2(tCO2)' 'WeekStatus' 'Load_Type']
The train score for Elastic Net model (alpha=0.0001) is 0.953152632387743
The train score for Elastic Net model (alpha=0.0001) is 0.9490660893682032


<h3>LinearSVR</h3>
<h4>with StandardScaler</h4>

In [282]:
scaler = StandardScaler()
features_train = scaler.fit_transform(features_train)

model = LinearSVR()

space = dict()
space['C'] = [0.1, 1, 10, 100]
space['fit_intercept'] = [True, False]
hyperparams_gridsearch = gridsearch(model, space)

model.fit_intercept = hyperparams_gridsearch['fit_intercept']
model.C = hyperparams_gridsearch['C']

selected_features = rfe_method(model)

features, target = selected_features, data['Usage_kWh']
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=0)

model.fit(features_train, target_train)
train_score = model.score(features_train, target_train)
test_score = model.score(features_test, target_test)

print("The train score for Linear SVR model (C=" + str(model.C) + ") is {}".format(train_score))
print("The train score for Linear SVR model (C=" + str(model.C) + ") is {}".format(test_score))

Grid Search - Best Score: -2.6621974557962487
Grid Search - Best Hyperparameters: {'C': 1, 'fit_intercept': True}


d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\P

Optimum number of features: 7
Score with 7 features: 0.910257


d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


['Lagging_Current_Reactive.Power_kVarh'
 'Leading_Current_Reactive_Power_kVarh' 'CO2(tCO2)'
 'Lagging_Current_Power_Factor' 'Leading_Current_Power_Factor'
 'WeekStatus' 'Load_Type']
The train score for Linear SVR model (C=1) is 0.9087969946418262
The train score for Linear SVR model (C=1) is 0.9083281812901561


d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


<h3>LinearSVR</h3>
<h4>with MinMaxScaler</h4>

In [283]:
scaler = MinMaxScaler()
features_train = scaler.fit_transform(features_train)

model = LinearSVR()

space = dict()
space['C'] = [0.1, 1, 10, 100]
space['fit_intercept'] = [True, False]
hyperparams_gridsearch = gridsearch(model, space)

model.fit_intercept = hyperparams_gridsearch['fit_intercept']
model.C = hyperparams_gridsearch['C']

selected_features = rfe_method(model)

features, target = selected_features, data['Usage_kWh']
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=0)

model.fit(features_train, target_train)
train_score = model.score(features_train, target_train)
test_score = model.score(features_test, target_test)

print("The train score for Linear SVR model (C=" + str(model.C) + ") is {}".format(train_score))
print("The train score for Linear SVR model (C=" + str(model.C) + ") is {}".format(test_score))

d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Grid Search - Best Score: -2.378067411577491
Grid Search - Best Hyperparameters: {'C': 10, 'fit_intercept': True}


d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\P

Optimum number of features: 1
Score with 1 features: 0.957907


d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


['CO2(tCO2)']
The train score for Linear SVR model (C=10) is 0.9612534698908565
The train score for Linear SVR model (C=10) is 0.9577393967563517


<h3>LinearSVR</h3>
<h4>with RobustScaler</h4>

In [284]:
scaler = RobustScaler()
features_train = scaler.fit_transform(features_train)

model = LinearSVR()

space = dict()
space['C'] = [0.1, 1, 10, 100]
space['fit_intercept'] = [True, False]
hyperparams_gridsearch = gridsearch(model, space)

model.fit_intercept = hyperparams_gridsearch['fit_intercept']
model.C = hyperparams_gridsearch['C']

selected_features = rfe_method(model)

features, target = selected_features, data['Usage_kWh']
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=0)

model.fit(features_train, target_train)
train_score = model.score(features_train, target_train)
test_score = model.score(features_test, target_test)

print("The train score for Linear SVR model (C=" + str(model.C) + ") is {}".format(train_score))
print("The train score for Linear SVR model (C=" + str(model.C) + ") is {}".format(test_score))

Grid Search - Best Score: -2.716662528701074
Grid Search - Best Hyperparameters: {'C': 1, 'fit_intercept': True}


d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\P

Optimum number of features: 6
Score with 6 features: 0.911644


d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


['Lagging_Current_Reactive.Power_kVarh'
 'Leading_Current_Reactive_Power_kVarh' 'CO2(tCO2)'
 'Lagging_Current_Power_Factor' 'WeekStatus' 'Load_Type']
The train score for Linear SVR model (C=1) is 0.9027746963597506
The train score for Linear SVR model (C=1) is 0.9007568207119518


d:\Programs\Python\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
